In [21]:
## 1. select later files of longitudial data as training set for harmonic wavelet computation.
## 2. put the remaining files as the test set, where spectrum (inner product of time_signal and wavelet) is calculated.

import glob
import re
import csv
import os
import shutil
import pandas as pd

# use corr matrix to select subjects as some of subject may have nan values in corr matrix and omitted.
input_dir = '/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix*/' 
input_dir2 = '/home/xin/Downloads/BrainImaging_UNC/out04_adni_roi_signals*/'
output_dir = 'adni_out01_time_signal_test_train_split'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    
if not os.path.isdir(output_dir + '/corr_matrix_training/'):
    os.mkdir(output_dir + '/corr_matrix_training/')

## for each subject, select the first session as test set and the last as the training set.
all_files = glob.glob(input_dir + '*.txt')
all_files

unique_subject = set()
for f in all_files:
    unique_subject.add(re.search('(.*)_sub-(.*)-ses(.*).txt', f).group(2))

unique_subject = list(unique_subject)
unique_subject.sort()

# for each subject, select the first session (roi signals) as the test set:
subject_test = []
subject_train = []

for sub in unique_subject:
    
    sub_file = glob.glob(input_dir + '*' + sub + '*.txt')
    sub_file2= glob.glob(input_dir2+ '*' + sub + '*.txt')
    subject_test.append(sub_file2[0])
    
    if len(sub_file)>2:
        # save corr matrix files to train set, we will use this to estimate harmonic wavelets with matlab codes.
        subject_train.append(sub_file[-1])
        
        # copy training set as a seperate directory:
        fname = re.search('(.*)/(.*)', sub_file[-1]).group(2)
        shutil.copy2(sub_file[-1], output_dir + '/corr_matrix_training/' + fname)        



In [27]:
# add diagnosis to subject ID:
input_file2 = 'adni_subject_info/MRI_information_All_Measurement.csv'
subject_info = pd.DataFrame({'subject_id': unique_subject})

demographic_data = pd.read_csv(input_file2)
g = demographic_data.groupby(['PTID']).first().reset_index()
df_merge = subject_info.merge(g[['PTID', 'AGE', 'PTGENDER', 'DX_bl', 'DX']], how = 'left', 
                              left_on = 'subject_id', right_on = 'PTID').drop(columns = 'PTID')
df_merge
df_merge.to_csv(output_dir + '/subject_info.csv')

In [14]:
with open(output_dir + '/test_set_roi_signals.csv', 'w', newline='') as myfile:
    # wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    # wr.writerow(subject_test)
    for line in subject_test:
        myfile.write(line)
        myfile.write('\n')
        
with open(output_dir + '/train_set_corr_matrix.csv', 'w', newline='') as myfile:
    # wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    # wr.writerow(subject_test)
    for line in subject_train:
        #print(line)
        myfile.write(line)
        myfile.write('\n')